## Aggregate Moving Violations Data

The purpose of this notebook is threefold:
> 1. Use the ArcGIS API for Python (API) to extract multiple data files of the Moving Violations dataset.

> 2. Combining them to one dataset using Pandas package of Python.

> 3. Using the API to publish the aggregated data as a web feature layer. 

In [1]:
#Importing packages, establishing connection
from arcgis.features import FeatureLayer
from arcgis.gis import GIS
gis = GIS("http://dcdev.maps.arcgis.com/", "username", "password")

In [2]:
moving_violations_source = 'http://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Violations_Moving_2016/MapServer/'

This is the server location for all the individual datasets for the Moving Violations data, one dataset for each month of the year 2016.
 
To extract the data, for a particular month we look within the folder number for that month

For instance

* Data for January 2016, can be extracted using the URL `http://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Violations_Moving_2016/MapServer/`__`0`__

* Data for February 2016, can be extracted using the URL `http://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Violations_Moving_2016/MapServer/`__`1`__

and so on..

Here, we demonstrate extracting data for the months of January-June 2016, and then aggregating them all in a single dataset.

If data for January can be found within foder 0, data for June can be found within data 5. To verify, we observe the number within parenthesis for the month we are looking up in the URL stored in `moving_violations_source`.

We then extract, convert and combine this data within a loop

In [3]:
import pandas as pd
all_months = pd.DataFrame()

In [4]:
for i in range(0,6):
    #builds url to extract data for specific month from Jan-June
    url = moving_violations_source + str(i)
    data_layer = FeatureLayer(url)
    #Extracts all data for specific month
    month_features = data_layer.query()
    #Converts it to a pandas dataframe
    month_data = month_features.df
    #Add the new month below the existing data in all_months
    all_months = pd.concat([all_months, month_data])

In [5]:
#(Row count, Column count)
all_months.shape

(532884, 19)

In [6]:
#Viewing the first 5 rows of the dataset
all_months.head()

,ACCIDENTINDICATOR,ADDRESS_ID,AGENCYID,FINEAMT,LOCATION,OBJECTID,PENALTY1,PENALTY2,ROW_,ROW_ID,STREETSEGID,TICKETISSUEDATE,TICKETTYPE,TOTALPAID,VIOLATIONCODE,VIOLATIONDESC,XCOORD,YCOORD,SHAPE
0,NO,805065,25,100,2200 BLK K ST NW W/B,11844743,0,0,6991794,None,4361.0,1451606400000,PHOTO,100.0,T119,SPEED 11-15 MPH OVER THE SPEED LIMIT,395664.920000,137186.890000,"{'x': 395664.92, 'y': 137186.89}"
1,NO,303037,25,100,100 BLK IRVING ST NW W/B,11844744,100,0,6991795,None,6350.0,1451606400000,PHOTO,200.0,T119,SPEED 11-15 MPH OVER THE SPEED LIMIT,398820.990000,140084.440000,"{'x': 398820.99, 'y': 140084.44}"
2,NO,805065,25,100,2200 BLK K ST NW W/B,11844745,0,0,6991796,None,4361.0,1451606400000,PHOTO,100.0,T119,SPEED 11-15 MPH OVER THE SPEED LIMIT,395664.920000,137186.890000,"{'x': 395664.92, 'y': 137186.89}"
3,NO,801296,25,150,DC295 NE AT BENNING RD EXIT N/B,11844746,0,0,6991797,None,5134.0,1451606400000,PHOTO,150.0,T120,SPEED 16-20 MPH OVER THE SPEED LIMIT,404307.500000,136594.240000,"{'x': 404307.5, 'y': 136594.24}"
4,NO,912878,2,100,SCOTT CIR NW @ 16TH ST NW,11844747,0,0,6991798,None,2405.0,1451606400000,MOVING,100.0,T185,MAKING A TURN FROM THE WRONG LANE,396831.551636,137709.369718,"{'x': 396831.55163636, 'y': 137709.36971762}"


We now extract this dataframe as a csv file that will be used to publish this data as a web layer.

We will first have to rename columns from the dataset, for them to be identified as geometry fields.

In [7]:
#renaming columns
all_months = all_months.rename(columns={'XCOORD': 'longitude', 'YCOORD': 'latitude'})
all_months.head()

,ACCIDENTINDICATOR,ADDRESS_ID,AGENCYID,FINEAMT,LOCATION,OBJECTID,PENALTY1,PENALTY2,ROW_,ROW_ID,STREETSEGID,TICKETISSUEDATE,TICKETTYPE,TOTALPAID,VIOLATIONCODE,VIOLATIONDESC,longitude,latitude,SHAPE
0,NO,805065,25,100,2200 BLK K ST NW W/B,11844743,0,0,6991794,None,4361.0,1451606400000,PHOTO,100.0,T119,SPEED 11-15 MPH OVER THE SPEED LIMIT,395664.920000,137186.890000,"{'x': 395664.92, 'y': 137186.89}"
1,NO,303037,25,100,100 BLK IRVING ST NW W/B,11844744,100,0,6991795,None,6350.0,1451606400000,PHOTO,200.0,T119,SPEED 11-15 MPH OVER THE SPEED LIMIT,398820.990000,140084.440000,"{'x': 398820.99, 'y': 140084.44}"
2,NO,805065,25,100,2200 BLK K ST NW W/B,11844745,0,0,6991796,None,4361.0,1451606400000,PHOTO,100.0,T119,SPEED 11-15 MPH OVER THE SPEED LIMIT,395664.920000,137186.890000,"{'x': 395664.92, 'y': 137186.89}"
3,NO,801296,25,150,DC295 NE AT BENNING RD EXIT N/B,11844746,0,0,6991797,None,5134.0,1451606400000,PHOTO,150.0,T120,SPEED 16-20 MPH OVER THE SPEED LIMIT,404307.500000,136594.240000,"{'x': 404307.5, 'y': 136594.24}"
4,NO,912878,2,100,SCOTT CIR NW @ 16TH ST NW,11844747,0,0,6991798,None,2405.0,1451606400000,MOVING,100.0,T185,MAKING A TURN FROM THE WRONG LANE,396831.551636,137709.369718,"{'x': 396831.55163636, 'y': 137709.36971762}"


In [8]:
#saving this to a csv file
all_months.to_csv('moving_violations_2016.csv')

In [9]:
# add the initial csv file and publish that as a web layer
item_prop = {'title':'Moving Violations for January-June 2016'}
csv_item = gis.content.add(item_properties=item_prop, data='moving_violations_2016.csv')
csv_item

<Item title:"Moving Violations for January-June 2016" type:CSV owner:mmajumdar_dcdev>

In [10]:
# publish the csv item into a feature layer
data_layer = csv_item.publish()
data_layer

<Item title:"Moving Violations for January-June 2016" type:Feature Layer Collection owner:mmajumdar_dcdev>

In [11]:
#sharing it with 'dcdev' organization
data_layer.share(True)

{'itemId': '0b1bbd68eec94e8faea018e041261a2d', 'notSharedWith': []}